In [1]:
import numpy as np
import pygmsh

from scar.geometry.Geometry2D import *

In [2]:
def dist(pt1,pt2):
    return np.sqrt((pt1[0]-pt2[0])**2+(pt1[1]-pt2[1])**2)

def tri(c_t):
    n_bc_points = c_t.shape[0]
    c_t_tri = [c_t[0,:]]
    c_t = np.delete(c_t,0,axis=0)
    for _ in range(1,n_bc_points):
        pt1 = c_t_tri[-1]
        # point le plus proche de pt1
        dist_pts = [dist(pt1,c_t[i,:]) for i in range(len(c_t))]
        i_min = np.argmin(dist_pts)
        c_t_tri.append(c_t[i_min,:])
        c_t = np.delete(c_t,i_min,axis=0)
    return np.array(c_t_tri)

In [3]:
form = Bean()

n_bc_points = 100
t = np.linspace(0,1,n_bc_points)
c_t = form.c(t)
c_t = np.array(c_t.T)
c_t = c_t[:-1,:]

c_t_tri = tri(c_t)
# c_t_tri = np.concatenate([c_t_tri,np.array([c_t_tri[0]])],axis=0)

/home/flecourtier/Bureau/Thèse/src/scar/geometry/Geometry2D.py:147: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400441250/work/torch/csrc/utils/tensor_new.cpp:261.)
  return self.R()@torch.Tensor([x,y])


In [5]:
# # plot polygon (sorted and unsorted)
# fig,axs = plt.subplots(1,2,figsize=(10,5))
# for i in range(n_bc_points-1):
#     axs[0].plot([c_t[i,0],c_t[i+1,0]],[c_t[i,1],c_t[i+1,1]],"black",linewidth=3)
#     axs[1].plot([c_t_tri[i,0],c_t_tri[i+1,0]],[c_t_tri[i,1],c_t_tri[i+1,1]],"black",linewidth=3)
# axs[1].plot([c_t_tri[-2,0],c_t_tri[-1,0]],[c_t_tri[-2,1],c_t_tri[-1,1]],"black",linewidth=3)

In [6]:
# create polygonal domain with pygmsh

with pygmsh.geo.Geometry() as geom:

    # poly1 = geom.add_polygon([
    #     [0.0, 0.0, 0.0],
    #     [0.0, 1.0, 0.0],
    #     [1.0, 1.0, 0.0],
    #     [1.0, 0.0, 0.0],
    #     ],
    #     mesh_size=0.1
    # )
    geom.add_polygon(
        c_t_tri,
        mesh_size=0.5,
    )
    mesh = geom.generate_mesh()

# with pygmsh.geo.Geometry() as geom:
#     geom.add_circle([0.0, 0.0], 1.0, mesh_size=0.2)
#     mesh = geom.generate_mesh()


In [7]:
mesh.write("test_bean.vtk")